In [3]:
import pickle as pkl
from copyreg import pickle

import pandas as pd
import numpy as np
from scipy.ndimage import histogram

df = pd.read_csv("Churn_Modelling.csv")
X = df.drop(["Exited",'RowNumber','CustomerId','Surname'],axis=1)
y = df["Exited"]
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder = pkl.load(file)
with open("onehot_encoder_geo.pkl","rb") as file:
    onehot_encoder = pkl.load(file)
with open("scaler.pkl","rb") as file:
    scaler = pkl.load(file)

X["Gender"] = label_encoder.transform(X["Gender"])
geo_encoder = onehot_encoder.transform(X[["Geography"]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder.get_feature_names_out(['Geography']))
X=pd.concat([X.drop('Geography',axis=1),geo_encoded_df],axis=1)
# 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


/Users/wassim/Documents/Data Science/Projects/Churn Prediction ANN/Churn Prediction ANN/my_venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/wassim/Documents/Data Science/Projects/Churn Prediction ANN/Churn Prediction ANN/my_venv/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/wassim/Documents/Data Science/Projects/C

## ANN Implementation

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

In [13]:
model = Sequential([
    Input((X_train.shape[-1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])

In [17]:
log_dir = "log/fit" + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [19]:
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True) #stop training if improvement is minimal during epoch training

In [21]:
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8042 - loss: 0.4458 - val_accuracy: 0.8610 - val_loss: 0.3465
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8552 - loss: 0.3598 - val_accuracy: 0.8585 - val_loss: 0.3484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8648 - loss: 0.3318 - val_accuracy: 0.8580 - val_loss: 0.3496
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8607 - loss: 0.3434 - val_accuracy: 0.8620 - val_loss: 0.3470
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8623 - loss: 0.3445 - val_accuracy: 0.8530 - val_loss: 0.3472
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8656 - loss: 0.3349 - val_accuracy: 0.8535 - val_loss: 0.3430
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8737 - loss: 0.3191 - val_accuracy: 0.8590 - val_loss: 0.3456
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8643 - loss: 0.3282 - val_accu

In [22]:
model.save("model.h5")

In [26]:
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
%tensorboard --logdir logs/fit20241109-185046

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.